# Studying the investment history and performance of Senators

As an international student, I have always found it puzzling that congressmen/congresswomen and senators could invest in companies that they have significant influence over with minimal levels of disclosure. In this project, I aim to answer two broad questions:

- Do Senators actually outperform the market?

- Can we detect potential conflicts in interests in Senators' trading activities?

## Gathering Data Part I
We will need two sets of data: 

a. Investment activity of Senators

b. Stock prices on relevant buying/selling dates

### STOCK Act

In 2012, the STOCK Act passed in the Senate 96-3 and in the House 417-2. Because of the STOCK Act, politicians are now required to file and disclose their investment activities (stocks, bond, commodities futures, and other securities)within 45 days. These disclosure reports can be accessed at:
- [House] https://disclosures-clerk.house.gov/PublicDisclosure/FinancialDisclosure 

- [Senate] https://efdsearch.senate.gov/search/home/

It is possible to query the database(s) directly, and download each disclosure report individually. However, due to a latter amendment, electronic forms were made optional. It is also no longer possible to search or sort the data directly, as each investment is filed separately. Often, politician also make use of **handwritten** forms (scanned), making it difficult to parse out the actual contents of the disclosure.

Example of handwritten form: https://efdsearch.senate.gov/search/view/paper/5C666F29-7055-461D-B4F7-5EA73AFCD860/

### API

Instead, I will be using a dataset available at:

- https://senatestockwatcher.com/api **[2014 - 2023 trading activity]**

This is done for three reasons:

- It is a lot faster than querying the government database and getting IP banned (even with the use of sleep functions)!

- Do not have to perform OCR on PDFs of disclosure reports

- (Some) Handwritten forms are transcribed manually by volunteers 




In [ ]:
import requests
import json
import pandas as pd
import pickle

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
senate_link = "https://senate-stock-watcher-data.s3-us-west-2.amazonaws.com/aggregate/all_transactions.json"

In [ ]:
# Since the datasets are in JSON format, we need to flatten them into normal dataframe format
senate_response = requests.get(senate_link)
senate_json = json.loads(senate_response.text)
senate_df = pd.json_normalize(senate_json)

In [ ]:
# Let us take a look at the Senate dataframe
senate_df

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,party,state,industry,sector,senator,ptr_link,disclosure_date
0,02/10/2023,Joint,SCHW,Charles Schwab Corporation (The) Common Stock,Stock,Sale (Partial),"$1,001 - $15,000",--,Republican,AK,Investment Bankers/Brokers/Service,Finance,Dan Sullivan,https://efdsearch.senate.gov/search/view/ptr/5...,03/16/2023
1,02/21/2023,Joint,CHD,"Church &amp; Dwight Company, Inc. Common Stock",Stock,Purchase,"$1,001 - $15,000",--,Republican,AK,Package Goods/Cosmetics,Basic Industries,Dan Sullivan,https://efdsearch.senate.gov/search/view/ptr/5...,03/16/2023
2,02/10/2023,Joint,CHD,"Church &amp; Dwight Company, Inc. Common Stock",Stock,Purchase,"$1,001 - $15,000",--,Republican,AK,Package Goods/Cosmetics,Basic Industries,Dan Sullivan,https://efdsearch.senate.gov/search/view/ptr/5...,03/16/2023
3,02/10/2023,Joint,GOOG,Alphabet Inc. - Class C Capital Stock,Stock,Purchase,"$1,001 - $15,000",--,Republican,AK,"Computer Software: Programming, Data Processing",Technology,Dan Sullivan,https://efdsearch.senate.gov/search/view/ptr/5...,03/16/2023
4,03/14/2023,N/A,N/A,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,N/A,Unknown,,NaN,NaN,NaN,NaN,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/4...,03/14/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8298,08/17/2012,N/A,N/A,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,N/A,Unknown,,NaN,NaN,NaN,NaN,Sheldon Whitehouse,https://efdsearch.senate.gov/search/view/paper...,08/17/2012
8299,08/16/2012,N/A,N/A,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,N/A,Unknown,,NaN,NaN,NaN,NaN,Pat Roberts,https://efdsearch.senate.gov/search/view/paper...,08/16/2012
8300,08/15/2012,N/A,N/A,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,N/A,Unknown,,NaN,NaN,NaN,NaN,Rob Portman,https://efdsearch.senate.gov/search/view/paper...,08/15/2012
8301,08/02/2012,N/A,N/A,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,N/A,Unknown,,NaN,NaN,NaN,NaN,Thomas R. Carper,https://efdsearch.senate.gov/search/view/paper...,08/02/2012


## Cleaning Data Part I

In [ ]:
senate_df["asset_type"].value_counts()

Stock                   6340
PDF Disclosed Filing     798
                         535
Other Securities         310
Stock Option             308
Corporate Bond             9
Cryptocurrency             3
Name: asset_type, dtype: int64

For this project, we will only focus on the trading activities pertaining to stocks (public equities). Two reasons:

- Difficult to calculate returns for non-stock instruments and benchmark their relative performance

- Difficult for members of the public to emulate trades involving complex instruments (options, futures, etc)

In [ ]:
senate_df["type"].value_counts()

Purchase          3659
Sale (Full)       2076
Sale (Partial)    1692
N/A                798
Exchange            78
Name: type, dtype: int64

To calculate investment performance, we want to track investments that have been bought and sold in completion (i.e closed) . To do so, we have to track ticker names that have been sold completely (Sale (Full)) by each politician.

In [ ]:
# Cleaning the data
# First, we split the dataframe into different dataframes, each df represents one senator/representative
senate_stocks_df = senate_df[senate_df["asset_type"] == "Stock"]
senator_names = senate_stocks_df["senator"].unique()

all_senators_dict = dict()
for name in senator_names:
  all_senators_dict[name] = senate_stocks_df[senate_stocks_df["senator"] == name].copy()

In [ ]:
# Next, we need to track the investments that have been closed
def select_sold_stocks(all_politicians_dict):
  """
  Input:
    - all_politicians_dict: dictionary of dataframes, each df corresponds
                            to 1 elected politician

  Output:
    - all_politicians_dict: dictionary of dataframes, each df corresponds to 1 
                            elected politician with only investment transactions
                            that have been fully sold
  """
  match_on = "Sale (Full)"

  for politician in all_politicians_dict:
    investments_df = all_politicians_dict[politician]
    
    # track only transactions involving investments that have been sold
    sold_investments = investments_df[investments_df["type"] == match_on]["ticker"].unique()
    filtered_investments_df = investments_df[investments_df["ticker"].isin(sold_investments)]

    # Sometimes, politicians may short (sell) stock without a history of buying the stock
    # This could be due to: inheriting stocks from family members, bought stocks before disclosure was needed, ...
    # A VERY crude way of adjusting for this (for now) is to only keep investments where transaction history (count) > 2
    freq = filtered_investments_df['ticker'].value_counts()
    ticker_gt_2 = freq[freq>2].index
    transactions_gt_2 = filtered_investments_df[filtered_investments_df["ticker"].isin(ticker_gt_2)]
    
    # update dictionary
    all_politicians_dict[politician] = transactions_gt_2
  
  return all_politicians_dict

filtered_senate_dict = select_sold_stocks(all_senators_dict)

## Gathering Data Part II

Now, we have to approx the price in which politicians bought and sold their stocks. We will use the intra-day closing price as a proxy for the price that politicians bought and sold their stocks.

We will be using the yfinance API (based off Yahoo! Finance) to obtain the prices.

https://pypi.org/project/yfinance/



In [ ]:
!pip install yfinance
import yfinance as yf
import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Function to find the adjusted closing stock price (input)
import datetime

def get_adj_close_price(stock_ticker, date_obj):
  date_str = date_obj.strftime('%Y-%m-%d')
  next_date = date_obj + datetime.timedelta(days=1)
  next_date_str = next_date.strftime('%Y-%m-%d')
  ticker = yf.Ticker(stock_ticker)
  
  try:
    adj_close_price = ticker.history(start=date_str, end=next_date_str)["Close"][0]

  except: # to catch delisted stocks
    adj_close_price = "delisted"

  return adj_close_price


# Cleaning Data Part II

Now, we need to calculate the returns for each investment, of each politician.

There are some (many!) **ambiguities** around this. A few of the more pertinent ones are noted below:

    a. Some stocks have delisted and others have changed their ticker names (FB --> META).
    b. The exact investment amount is unknown; only ranges are given.
    c. Politicians could enter into a new position after closing a previous position in that stock (buy 10k of AAPL, sell 10k of AAPl, buy 10k of AAPL again).
    d. Exact buy/sell prices are not given

For this project, I will be calculating returns in the following manner to address the above ambiguities:

    a. Stocks that are delisted are not considered as part of returns calculation 
    b. Convert the Categorical Variable "amount", into a Quantitative Variable by identifying the median amount within each range
    c. Amount transacted for each investment is approximated by total amount sold
    d. Transaction prices are approximated by the intra-day closing price on day of transaction
    e. Buy/sell price is the weighted average sell price of all buy/sell transactions
    f. Investment period for one stock is given by the earliest purchase date and latest sell by date.
  


In [ ]:

def calculate_returns(all_politicians_dict):
  """
  Input: 
    - all_politicians_dict: dictionary of dataframes, each correspond to 1 
                            elected politician (Senators)

  Output: 
    - investment_log: list of dictionaries, each dictionary represents 1 investment
                      by 1 politician
  """
  amount_convert_dict = {"$1,001 - $15,000": 8000,
                         "$15,001 - $50,000": 32500,
                         "$50,001 - $100,000": 75000,
                         "$100,001 - $250,000": 175000,
                         "$250,001 - $500,000": 375000,
                         "$500,001 - $1,000,000": 750000,
                         "$1,000,001 - $5,000,000": 3000000,
                         "$5,000,001 - $25,000,000": 15000000,
                         "$25,000,001 - $50,000,000": 37500000}
  
  investment_log = []

  # first, we want to iterate over each each politician
  for politician in all_politicians_dict:
    politician_df = all_politicians_dict[politician]
    # convert "amount" into a quantitative variable; transaction_date into datetime object
    politician_df["median_amount"] = politician_df["amount"].replace(amount_convert_dict)

    politician_df['transaction_date'] = pd.to_datetime(politician_df['transaction_date'])

    # create a df for each ticker
    tickers = [x for _, x in politician_df.groupby('ticker')]

    # iterate over each ticker
    for ticker in tickers:
      ## check if ticker has been delisted
      ticker["price"] = ticker.apply(lambda row:get_adj_close_price(row["ticker"], row["transaction_date"]), 
                                    axis=1)
      if "delisted" in set(ticker["price"]) or not set(ticker["price"]):
        continue

      # Get latest full sale date: later/more recent dates are "larger"
      ## check if there is sell transaction first
      if "Sale (Full)" not in set(ticker["type"]) or not set(ticker["type"]):
        continue
      latest_full_sale_date = ticker[ticker["type"].isin(["Sale (Full)"])]["transaction_date"].max().strftime('%Y-%m-%d')

      # discard all purchase transactions after latest full sale date
      ## check if there is buy transaction first
      if "Purchase" not in set(ticker["type"]) or not set(ticker["type"]):
        continue
      ticker_cleaned = ticker[ticker["transaction_date"] <= latest_full_sale_date]

      ## check if there is/are buy transaction(s) left
      if "Purchase" not in set(ticker_cleaned["type"]) or not set(ticker_cleaned["type"]):
        continue
      earliest_purchase_date = ticker_cleaned[ticker["type"].isin(["Purchase"])]["transaction_date"].min().strftime('%Y-%m-%d')
      
      amount_transacted = ticker_cleaned[~ticker_cleaned["type"].isin(["Purchase"])]["median_amount"].sum()

      # find average sale price [this includes partial sale]
      ticker_cleaned["cost_revenue"] = ticker_cleaned["median_amount"] * ticker_cleaned["price"]
      avg_sell_price = ticker_cleaned[~ticker_cleaned["type"].isin(["Purchase"])]["cost_revenue"].sum() / ticker_cleaned[~ticker_cleaned["type"].isin(["Purchase"])]["median_amount"].sum()
      
      # find average buy price
      avg_buy_price = ticker_cleaned[ticker_cleaned["type"].isin(["Purchase"])]["cost_revenue"].sum() / ticker_cleaned[ticker_cleaned["type"].isin(["Purchase"])]["median_amount"].sum()
      
      # calculate returns
      returns = (avg_sell_price - avg_buy_price) / avg_buy_price

      # calculate annulized returns
      days_held = (ticker_cleaned[ticker_cleaned["type"] == "Sale (Full)"]["transaction_date"].max() - ticker_cleaned[ticker_cleaned["type"] == "Purchase"]["transaction_date"].min()).days
      annualized_returns = (1+returns) ** (365/days_held) - 1

      # filter out information we want to preserve
      investment_info = {"senator": ticker["senator"].values[0],
                         "ticker": ticker["ticker"].values[0],
                         "party": ticker["party"].values[0],
                         "state": ticker["state"].values[0],
                         "industry": ticker["industry"].values[0],
                         "sector": ticker["sector"].values[0],
                         "earliest_purchase_date": earliest_purchase_date,
                         "latest_full_sale_date": latest_full_sale_date,
                         "amount_transacted": amount_transacted,
                         "avg_buy_price": avg_buy_price,
                         "avg_sell_price": avg_sell_price,
                         "returns": returns,
                         "annualized_returns": annualized_returns}

      # update investment_log
      investment_log += [investment_info]      
    
  return investment_log

In [ ]:
senate_final_ls = calculate_returns(filtered_senate_dict)
pickle.dump(senate_final_ls, open("/content/drive/MyDrive/Colab Notebooks/Final Project/senate_final_ls.pkl", "wb"))